# Определение уязвимых групп населения

## Итоговый проект 

## Часть 1. Формирование датафрейма

In [1]:
import numpy as np
import pandas as pd
import pickle
import regex as re

Данные формируются частично из предоставленных в проекте файлов, отчасти - из дополнительных источников.

Сведения берутся по 85 федеральным субъектам. Архангельская, Тюменская области и автономные округа в их составе рассматриваются по отдельности. Будем ориентироваться на данные 2016 - 2020 г. -  сведения по этому периоду представлены наиболее полно, и результаты будут более согласованными. В случае необходимости можно рассматривать другие периоды.

In [2]:
# функция для извлечения данных по федеральным субъектам и стандартизации их названий
def federal_subjects(
    df, standard_names, names_to_drop=None, region_col='region'):
    """Считывание данных по федеральным субъектам"""
       
    def get_standard_name(x):
        """вывод названий регионов в стандартном виде"""
    
        for key, value in standard_names.items():
            x = re.sub('[\(].*?[\)]', '', x)  
            if re.search(key, x.lower()):
                return value
        return None
    
    if names_to_drop is None:
        names_to_drop = []
    df = df[~df[region_col].isin(names_to_drop)].copy()
    df[region_col] = df[region_col].apply(get_standard_name)
    return (df.dropna(subset=region_col)
            .set_index(region_col).sort_index()) 

### 1.1. Денежные доходы


Данные представлены за период с 2015 по 2020 годы.
per_capita - Среднедушевые денежные доходы (в месяц), руб. 
real_income - Реальные денежные доходы, в процентах к предыдущему году (покупательная способность: с учетом роста цен)
nominal_salary - Среднемесячная номинальная начисленная заработная плата, руб.
real_salary - Реальная начисленная заработная плата, в процентах к предыдущему году (покупательная способность: с учетом роста цен)


In [3]:
def incomes_readout(file, sheet):
    df = pd.read_excel(file, sheet_name=sheet)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = federal_subjects(df, standard_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df
    

with open('data/standard_names.pkl', 'rb') as f:
    standard_names = pickle.load(f)
file = './data/cash_real_income_wages_2015_2020.xlsx'
per_capita = incomes_readout(file, 0)
real_income = incomes_readout(file, 1)
nominal_salary = incomes_readout(file, 2)
real_salary = incomes_readout(file, 3)

display('Среднедушевые денежные доходы (в месяц), руб.', per_capita.head(3))
display('Реальные денежные доходы, в процентах к предыдущему году', real_income.head(3))
display('Среднемесячная номинальная начисленная заработная плата, руб.', nominal_salary.head(3))
display('Реальная начисленная заработная плата, в процентах к предыдущему году', real_salary.head(3))

'Среднедушевые денежные доходы (в месяц), руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20860.0,21256.0,22139.0,22829.0,23937.0,23864.0
Амурская область,28240.0,27976.0,29213.0,30937.0,33304.0,35499.0
Архангельская область без Ненецкого АО,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0


'Реальные денежные доходы, в процентах к предыдущему году'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,99.1,94.7,100.0,99.7,99.6,95.5
Амурская область,96.1,92.1,101.1,102.4,101.7,100.3
Архангельская область без Ненецкого АО,95.1,93.0,98.7,101.7,100.2,98.4


'Среднемесячная номинальная начисленная заработная плата, руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20090.0,21202.0,22743.0,25519.0,27962.0,30072.0
Амурская область,32902.0,33837.0,37368.0,42315.0,47234.0,52430.0
Архангельская область без Ненецкого АО,35592.0,38118.0,40352.0,45427.0,49435.0,52779.0


'Реальная начисленная заработная плата, в процентах к предыдущему году'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,90,98.4,103.6,109.3,104.9,103.8
Амурская область,88,96.0,107.4,110.1,106.0,105.2
Архангельская область без Ненецкого АО,92.4,100.0,102.5,110.7,104.0,102.9


### 1.2. Детская смертность

Данные представлены за период с 1990 по 2021 годы. Единица измерения - количество человек. Берем суммарные данные по городской и сельской статистике. Данные по Архангельской и Тюменской областям пересчитываем самостоятельно из-за различающейся организации данных в зависимости от года. 

In [4]:
def mortality_readout(file):
    df = pd.read_excel(file, header=2, skipfooter=13, usecols='A, D:AI')
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    names_to_drop = [
        '            Архангельская область (кроме Ненецкого автономного округа)',
        '            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    ]
    df = federal_subjects(df, standard_names, names_to_drop)
    df.loc['Архангельская область без Ненецкого АО'] \
        = (df.loc['Архангельская область без Ненецкого АО']
           - df.loc['Ненецкий автономный округ'])
    df.loc['Тюменская область без округов'] \
        = (df.loc['Тюменская область без округов']
           - df.loc['Ханты-Мансийский автономный округ - Югра']
           - df.loc['Ямало-Ненецкий автономный округ'])
    df.columns = pd.to_datetime(df.columns, format='%Y').year
    return df


rural = mortality_readout(
    'data/child_mortality_rural_1990_2021.xls')
urban = mortality_readout(
    'data/child_mortality_urban_1990_2021.xls')
child_mortality = rural.add(urban)
display('Число умерших на первом году жизни', child_mortality.head(3))


'Число умерших на первом году жизни'

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,522.0,567.0,529.0,560.0,497.0,490.0,405.0,375.0,340.0,391.0,...,330.0,290.0,275.0,222.0,211.0,185.0,176.0,114.0,97.0,88.0
Амурская область,343.0,293.0,280.0,244.0,242.0,250.0,245.0,268.0,246.0,270.0,...,161.0,119.0,101.0,83.0,53.0,48.0,46.0,43.0,42.0,36.0
Архангельская область без Ненецкого АО,288.0,274.0,245.0,237.0,224.0,206.0,194.0,157.0,167.0,168.0,...,108.0,113.0,97.0,84.0,79.0,72.0,53.0,58.0,29.0,32.0


### 1.3. Инвалидность среди трудоспособного возраста

Данные представлены за период с 2017 по 2022 годы. Единица измерения - количество человек. Данные представлены по месяцам и для разных возрастов. Исключим из рассмотрения данные по населению пенсионного возраста: количество инвалидов пожилого возраста может зависеть и от естественного старения. Инвалидность среди трудоспособного населения даст нам более точную оценку социального неблагополучия региона.  

In [5]:
disabled = pd.read_csv(
    'data/disabled_total_by_age_2017_2022.csv')
disabled = federal_subjects(disabled, standard_names)

disabled['date'] = pd.to_datetime(disabled['date'])
disabled['dayofyear'] = disabled['date'].dt.dayofyear
disabled = disabled[disabled['dayofyear'] == 1]
disabled['young'] = disabled[['18_30', '31_40', '41_50', '51_60']].sum(axis=1)
disabled_18_60 = pd.pivot_table(
    data=disabled, values='young', index='region', columns='date')
disabled_18_60.columns = (disabled_18_60.columns).year
display('Инвалидность среди трудоспособного возраста', disabled_18_60.head(3))


'Инвалидность среди трудоспособного возраста'

date,2017,2018,2019,2020,2021,2022
region,,,,,,
Алтайский край,66770.0,64605.0,62840.0,61378.0,59810.0,59006.0
Амурская область,27892.0,26720.0,25976.0,25069.0,24149.0,23356.0
Архангельская область без Ненецкого АО,29228.0,28262.0,27615.0,26984.0,26498.0,26009.0


### 1.4. Алкоголизм, наркомания

Данные представлены за период с 2005 по 2018 годы.  Единица измерения - количество акогольных/наркотических психозов на 100 тыс. населения.

In [6]:
def alco_readout(file, sheet):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    df = federal_subjects(df, standard_names)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df


file = 'data/drug_alco.xlsx'
alco = alco_readout(file, 0)
alco1718 = alco_readout(file, 1)
drug = alco_readout(file, 2)
drug1718 = alco_readout(file, 3)
alco = pd.concat([alco, alco1718], axis=1).sort_index()
drug = pd.concat([drug, drug1718], axis=1).sort_index()

display('Заболеваемость с впервые в жизни установленным диагнозом алкоголизма и алкогольного психоза на 100 тыс.  населения', alco.head(3))
display('Заболеваемость с впервые в жизни установленным диагнозом наркомании на 100 тыс.  населения', drug.head(3))

'Заболеваемость с впервые в жизни установленным диагнозом алкоголизма и алкогольного психоза на 100 тыс.  населения'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область без Ненецкого АО,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7


'Заболеваемость с впервые в жизни установленным диагнозом наркомании на 100 тыс.  населения'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область без Ненецкого АО,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2


### 1.5. Валовый региональный продукт на душу населения

Данные представлены за период с 1996 по 2020 годы. 
Возьмем данные с 2011 года - в них меньше пропусков.

In [7]:
gross_regional_product = pd.read_excel('data/gross_regional_product_1996_2020.xls', 
                    header=2, skipfooter=1, usecols='A, Q:Z')
gross_regional_product.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']
gross_regional_product = federal_subjects(gross_regional_product, standard_names, names_to_drop)
gross_regional_product.columns = pd.to_datetime(gross_regional_product.columns).year
display('Валовой региональный продукт за год, предшествующий предыдущему, в текущих и постоянных ценах', gross_regional_product.head(3))



'Валовой региональный продукт за год, предшествующий предыдущему, в текущих и постоянных ценах'

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,
Алтайский край,137677.2,153556.7,173763.5,186798.6,204933.1,224525.8,231268.4,247599.3,271319.7,291156.9
Амурская область,273175.8,280023.9,258817.0,286282.6,343385.7,370192.4,373935.1,419905.2,521060.1,571362.1
Архангельская область без Ненецкого АО,232540.7,270662.9,283264.5,310817.4,352837.9,400764.6,441961.6,493205.1,509917.0,514200.4


### 1.6. Жилищные условия по данным 2020 г.

Данные представлены по домохозяйствам и их оценке своих жилищных условий.
Возьмем из этого: 
- Размер жилой площади в расчете на члена домохозяйства;
- процент домохозяйств, указавших на плохое или очень плохое состояние жилого помещения;
- процент домохозяйств, не имеющих средств для улучшения жилищных условий (рассчитывают на получение нового жилья в связи со сносом дома, собираются подать документы для постановки на очередь (и/или ожидают прохождения очереди))

In [8]:
def housing_readout(file, sheet, cols, skipfooter=0):
    df = pd.read_excel(
        file, sheet_name=sheet, usecols=cols, skipfooter=skipfooter)
    df.rename({'Регион': 'region'}, axis=1, inplace=True)
    
    # Ошибки в написании названий (латиница?)
    df.replace({'Bладимирская область': 'Владимирская'}, inplace=True)
    df.replace({'Bолгоградская область': 'Волгоградская'}, inplace=True)
    df.replace({'Bологодская область': 'Вологодская'}, inplace=True)
    df.replace({'Bоронежская область': 'Воронежская'}, inplace=True)
    df.replace('...', 0, inplace=True)  # слабая статистика -> 0
    df = federal_subjects(df, standard_names)
    return df


file = 'data/housing_2020.xlsx'
living_area = housing_readout(file, 0, 'A, H')
living_area.columns.values[0] = 'living area'
intents = housing_readout(file, 1, 'A, C, E, H, I', 2)
intents.columns.values[0] = 'to improve'
intents.columns.values[1] = 'bad conditions'
intents.columns.values[2] = 'queue'
intents.columns.values[3] = 'demolition'
intents['no funds'] = ((intents['to improve']/100 
    * (intents['queue']/100 + intents['demolition']/100)) * 100)
housing = pd.concat(
    [living_area, intents[['bad conditions', 'no funds']]], axis=1)
housing.head(3)

,living area,bad conditions,no funds
region,,,
Алтайский край,18.0,0.3,0.8642
Амурская область,17.3,1.3,1.0731
Архангельская область без Ненецкого АО,16.9,1.4,1.2998


### 1.7. Заболеваемость

 Данные представлены за период с 2005 по 2020 годы. Но данные позже 2016 г. полностью отсутствуют. 
Выберем данные по диагнозам, которые могут быть связаны с социальным неблагополучием(Общая заболеваемость, врожденные пороки развитияб психические расстройства и расстройства поведения, подростковая беременность)
Выозьмем данные за 2014, 2015 и 2016 годы

In [9]:
morbidity = pd.read_excel(
        'data/morbidity_2005_2020_age_disease.xls',
        header=2, usecols='A:C, M:O')
morbidity.rename(
    {'Unnamed: 0': 'region', 'Unnamed: 1': 'type', 'Unnamed: 2': 'age'}, 
    axis=1, inplace=True)
morbidity = morbidity.dropna(subset=['2014', '2015', '2016'], how='all')
morbidity = federal_subjects(morbidity, standard_names)

# Общая заболеваемость
mask = (
    (morbidity['type'] == 'Все заболевания')
    & (morbidity['age'] ==  'Всего')   
)
general_morbidity = morbidity[mask].copy()
general_morbidity.drop(columns=['type', 'age', '2014'], inplace=True)
general_morbidity.columns = pd.to_datetime(
    general_morbidity.columns).year

# Врожденные пороки развития
mask = (
    (morbidity['type'] == 'Врожденные аномалии (пороки развития), деформации и хромосомные нарушения')
    & (morbidity['age'] ==  'Всего')   
)
congenital_malformation = morbidity[mask].copy()
congenital_malformation.drop(columns=['type', 'age'], inplace=True)
congenital_malformation.columns = pd.to_datetime(
    congenital_malformation.columns).year

# Психические расстройства и расстройства поведения
mask = (
    (morbidity['type'] == 'Психические расстройства и расстройства поведения')
    & (morbidity['age'] ==  'Всего')   
)
psychosis = morbidity[mask].copy()
psychosis.drop(columns=['type', 'age'], inplace=True)
psychosis.columns = pd.to_datetime(
    psychosis.columns).year

# Подростковая беременность (до 14 лет)
mask = (
    (morbidity['type'] == 'Беременность, роды и послеродовой период')
    & (morbidity['age'] ==  '0-14 лет')   
)
teenage_pregnancy = morbidity[mask].copy()
teenage_pregnancy.drop(columns=['type', 'age'], inplace=True)
teenage_pregnancy.columns = pd.to_datetime(
    teenage_pregnancy.columns).year

display('Общая заболеваемость', general_morbidity.head(3))
display('Врожденные пороки развития', congenital_malformation.head(3))
display('Психические расстройства', psychosis.head(3))
display('Подростковая беременность', teenage_pregnancy.head(3))

'Общая заболеваемость'

,2015,2016
region,,
Алтайский край,109652.6,112764.7
Амурская область,83919.0,83412.0
Архангельская область без Ненецкого АО,101587.4,99842.1


'Врожденные пороки развития'

,2014,2015,2016
region,,,
Алтайский край,176.1,196.7,200.1
Амурская область,391.2,335.2,370.1
Архангельская область без Ненецкого АО,380.8,423.9,458.9


'Психические расстройства'

,2014,2015,2016
region,,,
Алтайский край,1454.0,1467.7,1272.1
Амурская область,512.4,485.3,446.7
Архангельская область без Ненецкого АО,646.0,693.5,685.3


'Подростковая беременность'

,2014,2015,2016
region,,,
Алтайский край,3.4,8.3,3.3
Амурская область,22.4,13.3,4.4
Архангельская область без Ненецкого АО,10.5,6.9,13.8


### 1.8. Рождаемость

Данные представлены за период с января 2006 по май 2022, помесячно. Нам необходимо преобразовать таблицу в годовое исчисление. 2022 год в расчет не берем, т.к данные неполные.


In [10]:
newborn = pd.read_csv(
    'data/newborn_2006_2022_monthly.csv.txt', sep=';', decimal=',')
newborn.drop(columns='Unnamed: 198',inplace=True)
newborn.rename({'Region': 'region'}, axis=1, inplace=True)
names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]
newborn = federal_subjects(newborn, standard_names, names_to_drop=names_to_drop)
newborn.loc['Архангельская область без Ненецкого АО'] \
    = (newborn.loc['Архангельская область без Ненецкого АО']
      - newborn.loc['Ненецкий автономный округ'])
newborn.loc['Тюменская область без округов'] \
    = (newborn.loc['Тюменская область без округов']
       - newborn.loc['Ханты-Мансийский автономный округ - Югра']
       - newborn.loc['Ямало-Ненецкий автономный округ'])
    
# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
newborn.drop(columns=2022, inplace=True)  # Неполный год
newborn = newborn.round(0).astype(int)
newborn.columns = pd.to_datetime(newborn.columns, format='%Y').year

display('Рождаемость',newborn.head(3))

'Рождаемость'

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,
Алтайский край,25513,28209,30711,31038,30863,30739,32695,32461,31429,30065,28867,25525,23491,21117,19976,18992
Амурская область,10405,10935,11233,11384,11473,11188,11733,11483,11159,10781,10377,9430,8889,7919,7853,7343
Архангельская область без Ненецкого АО,13396,14438,14577,14776,14806,14253,14730,14584,14331,13908,13313,11714,10671,9662,8906,8487


### 1.9. Население

Данные представлены за период с 1999 по 2022 с разбиением на регионы и муниципальные образования. Берем в расчет данные только по регионам.

In [11]:
population = pd.read_excel(
        'data/population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)

names_to_drop = [
        'Архангельская область (кроме Ненецкого автономного округа)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
        'Муниципальные образования Алтайского края',
        'Алтайский муниципальный район',
        'Алтайский сельсовет',
        'Большекалтайский сельсовет',
        'Населенные пункты, входящие в состав муниципальных образований Алтайского края',
        'Кировский сельсовет'
]
population = federal_subjects(population, standard_names, names_to_drop)
population.loc['Архангельская область без Ненецкого АО'] \
    = (population.loc['Архангельская область без Ненецкого АО']
      - population.loc['Ненецкий автономный округ'])
population.loc['Тюменская область без округов'] \
    = (population.loc['Тюменская область без округов']
       - population.loc['Ханты-Мансийский автономный округ - Югра']
       - population.loc['Ямало-Ненецкий автономный округ'])

newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = pd.to_datetime(newcols).year

display('Население',population.head(3))

'Население'

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2662738.0,2651628.0,2641079.0,2621050.0,2602595.0,2571987.0,2539430.0,2503510.0,2473024.0,2453455.0,...,2398751.0,2390638.0,2384812.0,2376774.0,2365680.0,2350080.0,2332813.0,2317153.0,2296353.0,2268179.0
Амурская область,949526.0,935607.0,923055.0,911381.0,901044.0,887781.0,874018.0,861056.0,850502.0,844290.0,...,816910.0,811274.0,809873.0,805689.0,801752.0,798424.0,793194.0,790044.0,781846.0,772525.0
Архангельская область без Ненецкого АО,1372631.0,1349160.0,1328187.0,1309552.0,1290956.0,1273668.0,1257312.0,1239924.0,1224813.0,1215264.0,...,1159506.0,1148760.0,1139950.0,1130240.0,1121813.0,1111031.0,1100290.0,1092424.0,1082662.0,1069782.0


### 1.10. Процент населения за чертой бедности (доходы ниже прожиточного минимума)

Данные представлены в процентах за период с 1992 по 2020 год. 1992-1994-Данные отсутствуют полностью по всем субъектам РФ. Их в расчет не берем.

In [12]:
poverty_percent = pd.read_csv(
    'data/poverty_percent_by_regions_1992_2020.txt')
poverty_percent = pd.pivot_table(
    data=poverty_percent, values='poverty_percent', 
    columns='year', index='region').reset_index()
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область'
]
poverty_percent = federal_subjects(poverty_percent, standard_names, names_to_drop)
poverty_percent.columns = pd.to_datetime(
    poverty_percent.columns, format='%Y').year

poverty_percent.drop(columns=[1992, 1993,1994], inplace=True)


poverty_percent.head()

year,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,33.7,46.8,45.7,52.9,53.8,53.9,47.3,38.9,33.9,30.9,...,22.6,20.6,17.6,17.1,18.0,17.8,17.5,17.4,17.6,17.5
Амурская область,36.1,28.2,26.3,31.2,38.0,47.7,45.3,44.6,35.6,33.8,...,20.4,16.0,16.2,14.8,15.2,17.0,16.7,15.6,15.7,15.2
Архангельская область без Ненецкого АО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3
Астраханская область,32.3,30.7,25.0,30.4,36.0,33.4,31.1,26.2,22.9,20.3,...,14.2,12.5,11.8,12.0,14.2,16.1,15.5,15.1,15.5,15.6
Белгородская область,19.9,17.6,18.9,20.1,24.2,33.6,29.1,25.4,22.9,20.7,...,8.6,6.5,7.6,7.7,8.5,8.1,7.8,7.5,7.8,7.2


### 1.10. Распределение малоимущего населения по социально-демографическим группам

Данные представлены за период с 2017 по 2020 годы. За 2019 и 2020 годы. данные частично не представлены (имеются пропуски или 0 во всех столбцах).  Во всех этих случаях возьмем данные по предшествующему году. 

In [13]:
def socdem_readout(file):
    df = pd.read_excel(file, header=2)
    df.rename({
        'Unnamed: 0': 'region',
        'Все население': 'all',
        'Дети в возрасте до 16 лет': 'children',
        'Население старше трудоспособного возраста': 'aged',
        'Население трудоспособного возраста': 'employable'
        }, axis=1, inplace=True)
    
    names_to_drop = [
        '        Архангельская область',
        '        Тюменская область'
    ]
    df = federal_subjects(df, standard_names, names_to_drop)
    return df


poverty_socdem_2017 = socdem_readout(
    'data/poverty_socdem_2017.xls')
poverty_socdem_2018 = socdem_readout(
    'data/poverty_socdem_2018.xls')
poverty_socdem_2019 = socdem_readout(
    'data/poverty_socdem_2019.xls')
poverty_socdem_2020 = socdem_readout(
    'data/poverty_socdem_2020.xls')

poverty_socdem = poverty_socdem_2020.copy()

mask = (poverty_socdem[['children', 'aged', 'employable']]
        .isna().all(axis=1))
poverty_socdem[mask] = poverty_socdem_2019

mask = ((poverty_socdem[['children', 'aged', 'employable']] == 0)
        .any(axis=1))
poverty_socdem[mask] = poverty_socdem_2018

mask = ((poverty_socdem[['children', 'aged', 'employable']] == 0)
        .any(axis=1))
poverty_socdem[mask] = poverty_socdem_2017

poverty_socdem.head(3)

,all,children,aged,employable
region,,,,
Алтайский край,100,31.4,9.2,59.4
Амурская область,100,38.4,4.7,56.9
Архангельская область без Ненецкого АО,100,28.5,7.2,64.3


### 1.11. Региональное производство

Здесь представлены данные за период с 2017 по 2020 годы по объему регионального производства/работам/услугам, распределенному по видам деятельности, в тысячах рублей. Нам необходимо получить суммарную инфо по всем видам производства по каждому региону.

In [14]:
reg_prod = pd.read_csv(
    'data/regional_production_2017_2020.txt')
names_to_drop = [
    '        Архангельская область', 
    '        Тюменская область']
reg_prod = federal_subjects(reg_prod, standard_names, names_to_drop) 
reg_prod.columns.values[-4:] = pd.to_datetime(reg_prod.columns[-4:]).year
reg_prod_total = (reg_prod
                  .groupby('region')[[2017, 2018, 2019, 2020]].sum()) 
reg_prod_total.columns = pd.to_datetime(
    reg_prod_total.columns, format='%Y').year

display('Региональное производство, по видам деятельности', reg_prod.head(5))
display('Региональное производство, общий объем', reg_prod_total.head(3))


'Региональное производство, по видам деятельности'

,production_field,2017,2018,2019,2020
region,,,,,
Алтайский край,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОР...",8068621.4,1.032303e+07,9813456.6,1.045441e+07
Алтайский край,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,6143029.4,5.023392e+06,7745963.3,9.146146e+06
Алтайский край,ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,307969929.4,3.228103e+08,357699013.3,3.618801e+08
Алтайский край,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПА...",45806615.7,4.743969e+07,43900541.6,4.474102e+07
Алтайский край,Промышленное производство (промышленность),367988195.7,3.855964e+08,419158974.7,4.262217e+08


'Региональное производство, общий объем'

,2017,2018,2019,2020
region,,,,
Алтайский край,735976391.6,7.711927e+08,838317949.5,8.524434e+08
Амурская область,268844147.6,2.687608e+08,340026685.8,4.045765e+08
Архангельская область без Ненецкого АО,709009752.2,6.064711e+08,533584941.4,6.641070e+08


### 1.12. Розничная торговля на душу населения

Данные представлены за период с 2000 по 2021 годы. В данных до 2013 года присутствуют пропуски. Возьмем период с 2013 по 2021

In [15]:
retail = pd.read_excel(
    'data/retail_turnover_per_capita_2000_2021.xls', 
    header=2, usecols='A, P:X')
retail.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
retail.drop(0, inplace=True)
names_to_drop = [
        '        Архангельская область',
        '            Архангельская область (без АО)',
        '        Тюменская область',
        '            Тюменская область (без АО)'
]
retail = federal_subjects(retail, standard_names, names_to_drop)
retail.columns = pd.to_datetime(retail.columns).year

retail.head(3)

,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Алтайский край,118096,128376,134925,137844,143873,150444,159514,153605,174321
Амурская область,145301,163781,182491,191523,202038,214688,231113,245233,276635
Архангельская область без Ненецкого АО,154177,176420,194345,202977,217332,229922,240155,250033,280050


### 1.17. Прожиточный минимум.

По данным Росстата(https://www.fedstat.ru/indicator/30957). Берем данные на 4 квартал 2020 г. для трудоспособного населения.

In [16]:
living_wage = pd.read_excel(
    'data/living_wage_2020.xls',
    header=4, usecols='A, I')
living_wage.rename({
    'Unnamed: 0': 'region',
    'Трудоспособное население.3': 'living wage',
    }, axis=1, inplace=True)
names_to_drop = ['        Архангельская область',
                 '        Тюменская область']
living_wage = federal_subjects(living_wage, standard_names, names_to_drop)
living_wage.head(3)

,living wage
region,
Алтайский край,10904.0
Амурская область,14331.0
Архангельская область без Ненецкого АО,14507.0


### 1.18.  Формирование итогового датасета.

Собираем датасет с данными преимущественно за период 2016-2020 гг. Усредняем значения

Описание финального датасета:
population-население, количество человек
per_capita- среднемесячный денежный доход на человека, в рублях
nominal_salary-среднемесячная номинальная начисленная заработная плата на человека, в рублях
child_mortality-детская смертность, в процентах от количества новорожденных.
disabled_18_60-инвалиды в возрасте 18-60 лет.
alco -количество впервые зарегистрированных алкогольных псизозов на 100 тыс. населения.
drug -количество впервые зарегистрированных наркотических псизозов на 100 тыс. населения.
gross_regional_product-валовый региональный продукт (объем произведенных товаров и услуг) на душу населения, в рублях
living_area-средняя жилая площадь на человека
bad_conditions-процент респондентов, указавших на плохое состояние жилья
no_funds-процент респондентов, рассчитывающих на получение жилья от государства
morbidity- общая заболеваемость на 100 тыс. населения, данные 2016 г.
congenital malformation-врожденные пороки развития на 100 тыс. населения.
psychosis-психические расстройства на 100 тыс. населения, данные 2016 г.
teenage_pregnancy-подростковая беременность на 100 тыс. населения, данные 2016 г.
birth_rate-рождаемость на 100 тыс. населения
poverty_percent-процент населения за чертой бедности
poor_children- процент детей среди малообеспеченного населения
poor_aged -процент пенсионеров среди малообеспеченного населения
poor_employable-процент людей трудоспособного возраста среди малообеспеченного населения
regional_production-региональное производство по всем видам деятельности на душу населения, в рублях
retail-розничная торговля на душу населения, в рублях
living_wage-прожиточный минимум, в рублях

In [17]:
population_mean = population.loc[:,'2016':'2020'].mean(axis=1)
per_capita_mean = per_capita.loc[:,'2016':'2020'].mean(axis=1)
nominal_salary_mean = nominal_salary.loc[:,'2016':'2020'].mean(axis=1)
child_mortality_mean = pd.DataFrame(child_mortality.loc[:,'2016':'2020'].mean(axis=1) / newborn.loc[:,'2016':'2020'].mean(axis=1) * 100)
disabled_18_60_mean = disabled_18_60.loc[:,'2016':'2020'].mean(axis=1)
alco_mean = alco.loc[:,'2016':'2020'].mean(axis=1)
drug_mean = drug.loc[:,'2016':'2020'].mean(axis=1)
general_morbidity_mean = general_morbidity.loc[:,'2015':'2016'].mean(axis=1)
congenital_malformation_mean = congenital_malformation.loc[:,'2014':'2016'].mean(axis=1)
psychosis_mean = psychosis.loc[:,'2015':'2016'].mean(axis=1)
teenage_pregnancy_mean = teenage_pregnancy.loc[:,'2015':'2016'].mean(axis=1)
gross_regional_product_mean = gross_regional_product.loc[:,'2016':'2020'].mean(axis=1)
housing_mean = housing
newborn_mean = newborn.loc[:,'2016':'2020'].mean(axis=1)
birth_rate_mean = pd.DataFrame(newborn_mean/population_mean*100000)
poverty_percent_mean = poverty_percent.loc[:,'2016':'2020'].mean(axis=1)
poverty_socdem = poverty_socdem.drop(columns='all')
poverty_socdem.rename({
        'children': 'poor_children',
        'aged': 'poor_aged',
        'employable': 'poor_employable',
        }, axis=1),
reg_prod_total_mean = reg_prod_total.loc[:,'2017':'2020'].mean(axis=1)
regional_production = pd.DataFrame(reg_prod_total_mean / population_mean)
retail_mean = retail.loc[:,'2016':'2020'].mean(axis=1)
living_wage_mean = living_wage



In [18]:
russia_regions = pd.concat([
    population_mean,
    per_capita_mean,
    nominal_salary_mean,
    child_mortality_mean,
    disabled_18_60_mean,
    alco_mean,
    drug_mean,
    general_morbidity_mean,
    congenital_malformation_mean,
    psychosis_mean,
    teenage_pregnancy_mean,
    gross_regional_product_mean,
    housing_mean,
    birth_rate_mean,
    poverty_percent_mean,
    poverty_socdem,
    regional_production,
    retail_mean,
    living_wage_mean
    ], 
    axis=1
).fillna(0)



In [19]:
russia_regions.columns=['population','per_capita','nominal_salary','child_mortality','disabled_18_60','alco','drug','general_morbidity','congenital_malformation','psychosis',
'teenage_pregnancy','gross_regional_product','living_area','bad_conditions','no_funds', 'birth_rate','poverty_percent','poor_children','poor_aged','poor_employable','regional_production','retail','living_wage']

In [20]:
russia_regions.to_csv('data/russia_regions.csv')
russia_regions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Алтайский край to Ярославская область
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   population               85 non-null     float64
 1   per_capita               85 non-null     float64
 2   nominal_salary           85 non-null     float64
 3   child_mortality          85 non-null     float64
 4   disabled_18_60           85 non-null     float64
 5   alco                     85 non-null     float64
 6   drug                     85 non-null     float64
 7   general_morbidity        85 non-null     float64
 8   congenital_malformation  85 non-null     float64
 9   psychosis                85 non-null     float64
 10  teenage_pregnancy        85 non-null     float64
 11  gross_regional_product   85 non-null     float64
 12  living_area              85 non-null     float64
 13  bad_conditions           85 non-null     float64
 14  no_

### Вывод

Проведен большой объем работы с файлами разных форматов, содержащих демографические и финасово-экономические, медицинские данные. Из них сформированы датасеты со стандартизованными названиями федеральных субъектов РФ (85). На основе полученных датасетов сформирована таблица по срезу 2016-2020 гг., которая будет использована далее. 